In [ ]:
!pip install -U sentence-transformers git+https://github.com/huggingface/transformers@v4.56.0-Embedding-Gemma-preview -q

After you have accepted the license, you need a valid Hugging Face Token to access the model.

In [ ]:
# Login into Hugging Face Hub
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
#batch triplet + random modifications
# from datasets import load_dataset, Dataset
# import re
# import random
# import numpy as np
# class DatasetProcessor:

#     def __init__(self, negative_count:int = 4, total_ratio_of_missing:float = 0.3):
#         self.country_pattern = r"county.*?\n"
#         self.patterns = (r"(.*?\n)", r"(\n.*? )", r"( .*?\n)", r"( .*? )") #patterns for removing random vals
#         self.negative_count = negative_count
#         self.total_ratio_of_missing = total_ratio_of_missing

#     def get_masked_lines(self, line:str):
#         """
#         random masking with total_ratio_of_missing and returns samples; |samples| = negative_count
#         """
#         output_lines = []
#         combs = re.findall(self.patterns[0], line)
#         del_count = int(len(combs) * self.total_ratio_of_missing)
#         if del_count == 0:
#             del_count = 1

#         for i in range(self.negative_count):
#             line_in_loop = str(line)
#             count_in_loop = del_count
#             while count_in_loop>0:
                
#                 random_pattern = random.choice(self.patterns)
#                 try:
#                     random_value_to_drop = random.choice(re.findall(random_pattern, line_in_loop))
#                     line_in_loop = re.sub(random_value_to_drop, "", line_in_loop, count=1)
#                     count_in_loop -=1
#                 except:
#                     continue

#             output_lines.append(line_in_loop)

#         return output_lines

#     def train_row_wise_changes(self, example, id):
#         """
#         removes county info and applies random masking
#         """

#         example["output_text"] = re.sub(self.country_pattern, "", example["output_text"])

#         return {
#             "text": [example["input_text"], example["output_text"]] + self.get_masked_lines(example["output_text"]),
#             "label": [id * (self.negative_count + 1)]*2 +list( id*(self.negative_count+ 1)+1 + np.arange(0, self.negative_count) )#unique ids for hard negatives
#         }
#     def test_row_wise_changes(self, example):
#         """
#         removes county info and applies random masking
#         """

#         example["output_text"] = re.sub(self.country_pattern, "", example["output_text"])
#         example["failed_text"] = self.get_masked_lines(example["output_text"])
#         return example

#     @staticmethod
#     def flatten_lists_of_lists(batch):
#         flattened_texts = []
#         flattened_labels = []

#         for texts_lol, labels_lol in zip(batch['text'], batch['label']):
#             flat_text = [item for item in texts_lol]
#             flattened_texts.extend(flat_text)

#             flat_label = [item for item in labels_lol]
#             flattened_labels.extend(flat_label)
#         return {'text': flattened_texts, 'label': flattened_labels}

#     # @staticmethod
#     # def flatten_test_of_lists(batch): #if batchtriplet
#     #     flattened_inputs = []
#     #     flattened_outputs = []
#     #     flattened_fails = []
#     #     #print(len(flattened_inputs), len(flattened_inputs), len(flattened_inputs))
#     #     for flat_input, flat_output, fails in zip(batch['input_text'],batch['output_text'], batch['failed_text']):
#     #         #flat_input = [item for item in input]
#     #         flattened_inputs.extend([flat_input]*len(fails))

#     #         #flat_output = [item for item in output]
#     #         flattened_outputs.extend([flat_output]*len(fails))

#     #         flat_fail = [item for item in fails]
#     #         flattened_fails.extend(flat_fail)

#     #     return {'input_text': flattened_inputs, 'output_text': flattened_outputs, "failed_text":flattened_fails}

#     @staticmethod
#     def flatten_test_of_lists(batch): #if contrastive
#         flattened_inputs = []
#         flattened_outputs = []
#         flattened_labels = []

#         for flat_input, flat_output, fails in zip(batch['input_text'],batch['output_text'], batch['failed_text']):

#             flattened_inputs.extend([flat_input]*(len(fails)+1))
#             flattened_output = [flat_output] + [item for item in fails]
#             flattened_outputs.extend(flattened_output)
#             flattened_labels.extend([1]*len([flat_output]) + [0]*len(fails))
            
#         return {'input_text': flattened_inputs, 'output_text': flattened_outputs, "failed_text":[None]*len(flattened_labels),"label":flattened_labels}
#     def process(self, dataset:Dataset):
#         """
#         train_test split, all data permutation applied with class methods
#         """
#         dataset = dataset.select_columns(['input_text', 'output_text', 'output_empty', 'label'])
#         dataset = dataset.filter(lambda example: (example["output_empty"] == False) & (example["label"] == 1))
#         dataset = dataset.select_columns(['input_text', 'output_text'])
#         #if contrastive
#         dataset = dataset.map(self.test_row_wise_changes)
#         #------
#         dataset = dataset["train"].train_test_split(test_size=0.2)
#         #------
#         new_dataset = dataset.map(self.flatten_test_of_lists, batched = True).select_columns(['input_text', 'output_text', 'label']) 
        
#         #if batchtriplet
#         #train_subset = dataset["train"].map(self.train_row_wise_changes, with_indices=True)
#         #train_subset = train_subset.select_columns(['text', 'label']) #if batchtriplet
#         #test_subset = dataset["test"].map(self.test_row_wise_changes)
#         #new_dataset = {"train": train_subset.map(self.flatten_lists_of_lists, batched=True), 
#                        #"test": test_subset.map(self.flatten_test_of_lists, batched=True),}
#         #new_dataset = new_dataset["train"].train_test_split(test_size=0.2)
        
#         return new_dataset

# ds_proc = DatasetProcessor(negative_count = 5, total_ratio_of_missing=0.2)
# raw_ds = load_dataset('json', data_files='/kaggle/input/nlp-corpus-haystack/dataset.jsonl')
# new_ds = ds_proc.process(raw_ds)

In [ ]:
# #if you lack of VRAM then you can't use large batchtriplet, instead we will small one. also youcan consider use contrastive with low batch (BYOL says that it's bad approach)
# def reorder_dataset_columns(original_text_list, original_label_list):

#     new_text = []
#     new_label = []
#     prev_X = None
#     prev_T = None

#     # Iterate over the data in chunks of 3
#     for i in range(0, len(original_label_list), 3):
#         text_chunk = original_text_list[i:i+3]
#         label_chunk = original_label_list[i:i+3]

#         if (i // 3) % 2 == 0:
#             label_chunk = [1, 1, 0]
#             prev_X = label_chunk[0]
#             prev_T = text_chunk[0]

#             new_label.extend(label_chunk)
#             new_text.extend(text_chunk)
            
#         else:            
#             if prev_X is None or prev_T is None:
#                 new_label.extend(label_chunk)
#                 new_text.extend(text_chunk)
#                 continue
                
#             for l_j, t_j in zip(label_chunk, text_chunk):
#                 # Append the constant label (prev_X) twice, then the element's label (l_j)
#                 new_label.extend([1, 1, 0])
                
#                 # Append the constant text (prev_T) twice, then the element's text (t_j)
#                 new_text.extend([prev_T, prev_T, t_j])

#     return new_text, new_label

# new_text_list, new_label_list = reorder_dataset_columns(new_ds["train"]["text"], new_ds["train"]["label"])

# new_ds["train"] = Dataset.from_dict({'text': new_text_list, 'label': new_label_list})

In [ ]:
from datasets import load_dataset
import numpy as np
raw_ds = load_dataset('json', data_files='/kaggle/input/nlp-corpus-haystack/dataset.jsonl')  #/kaggle/input/nlp-corpus-haystack/dataset.jsonl

def process_input(example):
    example["input_text"] += f'\nsection: {example["section"]},\ncounty: {example["county"]},\nparcel_id: {example["parcel_id"]}'
    #example["output_text"] = re.sub(r"county.*?\n", "", example["output_text"])
    return example

def process_ds(dataset):
    #dataset = dataset.select_columns(['input_text', 'output_text', 'output_empty', 'label'])
    #dataset = dataset.filter(lambda example: example["output_empty"] == False)
    dataset = dataset.map(process_input)
    dataset = dataset.select_columns(['input_text', 'output_text', 'label'])
    
    return dataset

cos_ds = process_ds(raw_ds["train"])

#train_test_split without data leakage
from sklearn.model_selection import train_test_split
from datasets import DatasetDict
ids = np.arange(len(cos_ds))[:-1:2]
train_id, test_id = train_test_split(
    ids,
    train_size=0.8,
    random_state=42
)

rng = np.random.default_rng(seed = 42)
train_id_adj = train_id + 1
train_id = np.append(train_id, train_id_adj)
rng.shuffle(train_id)
test_id_adj = test_id + 1
test_id  = np.append(test_id, test_id_adj)
rng.shuffle(test_id)
cos_alt_ds = DatasetDict({
    "train": cos_ds.select(train_id),
    "test": cos_ds.select(test_id)
})
cos_ds = cos_alt_ds
cos_eval = cos_ds["test"]
#

# cos_ds = cos_ds.train_test_split(test_size=0.2, seed = 42) #possible data leakage
# cos_eval = cos_ds["test"]

In [ ]:
print(np.sort(test_id)[:7])
np.any(np.isin(train_id, test_id))

In [ ]:
import random
random.seed(42)

def country_change(example):
    counties_to_sample = ['Autauga', 'Cook', 'Harris', 'Maricopa', 'San Diego', 'Orange', 'Collier', 'Kings', 'Dallas']
    states_to_sample = ['AL', 'IL', 'TX', 'AZ', 'CA', 'CA', 'FL', 'NY', 'TX']
    #change county in each row
    idx = list(range(len(counties_to_sample)))
    sample_id = random.choice(idx)
    example["input_text"] = example["input_text"].replace("Collier", counties_to_sample[sample_id])
    example["input_text"] = example["input_text"].replace("FL", states_to_sample[sample_id])

    #change county in output
    if "Collier" in example["output_text"]:
        example["output_text"] = example["output_text"].replace("Collier", counties_to_sample[sample_id])
        example["output_text"] = example["output_text"].replace("FL", states_to_sample[sample_id])

        if example["label"] == 1:
            if random.choice([0, 1, 2]) == 1:

                random_counties = idx.copy()
                random_counties.remove(sample_id)

                example["output_text"] = example["output_text"].replace("Collier", counties_to_sample[random.choice(random_counties)])
                example["output_text"] = example["output_text"].replace("FL", states_to_sample[random.choice(idx)])
                
                
                example["label"] =0

    return example
cos_ds["train"] = cos_ds["train"].map(country_change)

In [ ]:
# import torch
# from sentence_transformers import SentenceTransformer
#
# device = "cuda" if torch.cuda.is_available() else "cpu"
#
# model_id = "google/embeddinggemma-300M"
# model = SentenceTransformer(model_id).to(device=device)
# label_embeddings = model.encode((new_ds["train"]['input_text'][1], new_ds["train"]['output_text'][1]), prompt_name="STS")
# similarities = model.similarity(label_embeddings[0], label_embeddings[1])
# similarities

In [ ]:
#model.prompts

In [ ]:
# import wandb
# wandb.init(mode="disabled")

In [ ]:
from enum import Enum
import torch.nn.functional as F
class SiameseDistanceMetric(Enum):
    EUCLIDEAN = lambda x, y: F.pairwise_distance(x, y, p=2)
    MANHATTAN = lambda x, y: F.pairwise_distance(x, y, p=1)
    COSINE_DISTANCE = lambda x, y: 1 - F.cosine_similarity(x, y)
    CUSTOM_LOSS = lambda x, y: 1 - F.cosine_similarity(x, y) + torch.linalg.norm((x-y), ord = torch.inf, dim = 1)

In [ ]:
from sentence_transformers import losses
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers import (
    sampler,
    SentenceTransformer,
    SentenceTransformerModelCardData,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
)
from sentence_transformers.evaluation import BinaryClassificationEvaluator, TripletEvaluator
import torch
model = SentenceTransformer(
    "google/embeddinggemma-300m",
    model_card_data=SentenceTransformerModelCardData(
        language="en",
        license="apache-2.0",
        model_name="EmbeddingGemma-300m trained to measure coverage",
    ),
)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
# loss = losses.BatchAllTripletLoss( #only if vram is big enough!
#     model=model,
#     distance_metric=losses.BatchHardTripletLossDistanceFunction.cosine_distance,
#     #margin=0.2
# )

loss = losses.ContrastiveLoss(
    model=model,
    distance_metric=SiameseDistanceMetric.COSINE_DISTANCE,#.EUCLIDEAN, #.MANHATTAN,
    margin = 0.1,
    size_average = True,
)
# loss = losses.CosineSimilarityLoss(
#     model = model
# )

loss = losses.MatryoshkaLoss(
    model = model,
    loss = loss,
    matryoshka_dims = [768, 512, 256, 128]
)

run_name = "embeddinggemma-300m-haystack-0.8k_contrastive"
prompt_use = "STS"
args = SentenceTransformerTrainingArguments(
    report_to=None,
    output_dir=f"models/{run_name}",
    
    num_train_epochs=5,
    per_device_train_batch_size=3, # if you have enough vram, then set to |positive| + |semi_hard_negative| else change dataset structure and train objective
    per_device_eval_batch_size=3,
    gradient_accumulation_steps = 2,
    learning_rate=2e-5,
    warmup_ratio=0.05,
    fp16=True,
    bf16=False,
    batch_sampler= BatchSamplers.BATCH_SAMPLER, #GROUP_BY_LABEL, #batchtriplet
    prompts={
        #"text": prompt_use,
        "input_text": prompt_use,
        "output_text": prompt_use
    },
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=40,
    save_strategy="epoch",
    #save_steps=80,
)
# test_evaluator = TripletEvaluator(
#     anchors=new_ds["test"]["input_text"],
#     positives=new_ds["test"]["output_text"],
#     negatives=new_ds["test"]["failed_text"],
#     name="test",
# )
cos_evaluator = BinaryClassificationEvaluator(
    sentences1=cos_eval["input_text"],
    sentences2=cos_eval["output_text"],
    labels=cos_eval["label"],
    #for l2
    #similarity_fn_names = ["euclidean"],
)

In [ ]:
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=cos_ds["train"], #new_ds
    eval_dataset=None,
    loss=loss,
    evaluator=cos_evaluator,
)
trainer.train()

In [ ]:
# results = test_evaluator(model)
# print(results)

In [ ]:
#model = SentenceTransformer(model_name_or_path = "/kaggle/working/models/embeddinggemma-300m-haystack-0.8k_contrastive/checkpoint-400")

In [ ]:
binary_results = cos_evaluator(model)
binary_results

In [ ]:
sims = []
labels = cos_eval["label"]
for row in cos_eval:
    label_embeddings = model.encode((row['input_text'], row['output_text']), 
                                    prompt_name=prompt_use, 
                                    show_progress_bar = False,
                                    #if l2
                                    #convert_to_numpy = False
                                   )
    #for cos
    similarities = model.similarity(label_embeddings[0], label_embeddings[1])
    #for l2 also could just change atribute in model object that sets up defalut distance function for .similiarity()
    #similarities = torch.nn.functional.pairwise_distance(label_embeddings[0], label_embeddings[1])
    sims.append(similarities)

In [ ]:
sims = torch.tensor(sims)

In [ ]:
import numpy as np
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
fpr, tpr, thresholds =roc_curve(np.array(cos_eval["label"][:],dtype = float), sims.detach().cpu().numpy())
print(roc_auc_score(np.array(cos_eval["label"][:],dtype = float), sims.detach().cpu().numpy()))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(fpr, tpr)

In [ ]:
# final_output_dir = f"models/{run_name}/final"
# model.save_pretrained(final_output_dir)

model.push_to_hub("mancer146/embeddinggemma-300m-haystack-contrastive-thin-fixed", commit_message = "0_0")